#  World Bank Data Cleaning & Preparation for Tableau Dashboard
This notebook prepares a World Bank dataset for interactive visualizations in Tableau. We’ll extract, clean, and export the data for analysis of electricity access and financial inclusion across countries and time.

##  Step 0: Extract ZIP File

We extract the World Bank dataset archive so we can access the main CSV file for cleaning.

In [5]:
import zipfile
import os

# ZIP file name
zip_path = 'P_Data_Extract_From_World_Development_Indicators.zip'

# Extraction directory
extract_dir = 'world_bank_data'

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Files extracted to:", extract_dir)


Files extracted to: world_bank_data


#  Step 1: Inspect Extracted Files
files = os.listdir(extract_dir)
print(" Files in directory:", files)

In [7]:
files = os.listdir(extract_dir)
print("Files in directory:", files)

Files in directory: ['ed35928c-f2c5-48a4-b96d-b6e37064a0db_Data.csv', 'ed35928c-f2c5-48a4-b96d-b6e37064a0db_Series - Metadata.csv']


#  Step 2: Load the Dataset
import pandas as pd

In [8]:
#  Imports
import pandas as pd
import os

#  Config – Define data file path
DATA_DIR = "world_bank_data"
DATA_FILE = "ed35928c-f2c5-48a4-b96d-b6e37064a0db_Data.csv"
data_path = os.path.join(DATA_DIR, DATA_FILE)

#  Load dataset
df = pd.read_csv(data_path)

#  Confirm successful load
print("Data loaded. Shape:", df.shape)


Data loaded. Shape: (1601, 11)


#  Step 3: Clean Column Names

In [9]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('(', '', regex=False)
    .str.replace(')', '', regex=False)
)
df.columns.tolist()

['country_name',
 'country_code',
 'series_name',
 'series_code',
 '1990_[yr1990]',
 '2006_[yr2006]',
 '2022_[yr2022]',
 '1975_[yr1975]',
 '1991_[yr1991]',
 '2007_[yr2007]',
 '2023_[yr2023]']

##  Step 4: Explore and Clean Data  
We check the shape of the dataset, look for missing values, inspect data types, and check for duplicates.

In [26]:
# Check dataset shape
print("Shape:", df.shape)

#  Missing values
print("\nMissing values per column:\n", df.isnull().sum())

#  Data types
print("\nData types:\n", df.dtypes)

#  Check for duplicates
print("\nDuplicate rows:", df.duplicated().sum())

Shape: (1601, 11)

Missing values per column:
 country_name     3
country_code     5
series_name      5
series_code      5
1990_[yr1990]    5
2006_[yr2006]    5
2022_[yr2022]    5
1975_[yr1975]    5
1991_[yr1991]    5
2007_[yr2007]    5
2023_[yr2023]    5
dtype: int64

Data types:
 country_name     object
country_code     object
series_name      object
series_code      object
1990_[yr1990]    object
2006_[yr2006]    object
2022_[yr2022]    object
1975_[yr1975]    object
1991_[yr1991]    object
2007_[yr2007]    object
2023_[yr2023]    object
dtype: object

Duplicate rows: 2


##  Step 5: Remove Rows with Missing Key Info  
We drop rows that are missing the country or series name, since those are critical for analysis and visualization.

In [10]:
df_clean = df.dropna(subset=['country_name', 'series_name'])
print(" Cleaned rows. New shape:", df_clean.shape)

 Cleaned rows. New shape: (1596, 11)


##  Step 6: Reshape Data to Long Format  
We use `pd.melt()` to turn multiple year columns into one `year` column with a matching `value` column.

In [11]:
df_long = pd.melt(
    df_clean,
    id_vars=['country_name', 'series_name', 'series_code'],
    var_name='year',
    value_name='value'
)

df_long.head()

,country_name,series_name,series_code,year,value
0,Afghanistan,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,country_code,AFG
1,Afghanistan,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,country_code,AFG
2,Afghanistan,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,country_code,AFG
3,Afghanistan,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,country_code,AFG
4,Afghanistan,Account ownership at a financial institution o...,FX.OWN.TOTL.MA.ZS,country_code,AFG


##  Step 7: Clean and Convert Data Types  
We extract the year from column headers, convert it to datetime format, and convert the value column to numeric for analysis.

In [12]:
# Extract numeric year from strings like '2000 [YR2000]'
df_long['year'] = df_long['year'].str.extract('(\\d{4})')

# Convert to datetime format (year only)
df_long['year'] = pd.to_datetime(df_long['year'], format='%Y', errors='coerce')

# Convert values to numeric
df_long['value'] = pd.to_numeric(df_long['value'], errors='coerce')

df_long.dtypes

country_name            object
series_name             object
series_code             object
year            datetime64[ns]
value                  float64
dtype: object

##  Step 8: Remove Missing Values from Final Dataset  
We remove any rows where `year` or `value` is missing, so our charts and dashboards don’t break or mislead.

In [13]:
df_final = df_long.dropna(subset=['year', 'value'])
print(" Final dataset shape:", df_final.shape)
df_final.head()

 Final dataset shape: (2436, 5)


,country_name,series_name,series_code,year,value
1604,Albania,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1990-01-01,100.000000
1622,Andorra,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1990-01-01,100.000000
1634,Antigua and Barbuda,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1990-01-01,93.438635
1640,Argentina,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1990-01-01,78.894588
1652,Aruba,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,1990-01-01,100.000000


##  Step 9: Export Cleaned Dataset  
We save the final cleaned dataset to a CSV file that can be imported into Tableau or Power BI for visualization.

In [14]:
df_final.to_csv("world_bank_cleaned.csv", index=False)
print("Cleaned data exported as 'world_bank_cleaned.csv'")

Cleaned data exported as 'world_bank_cleaned.csv'


## 🧾 Step 10: Final Summary  
We completed a full data cleaning process and exported a structured dataset for development insights.  
This includes electricity access, financial inclusion, and other indicators across countries from 2006 to 2022.  
The cleaned dataset is now ready for analysis in Tableau or Power BI.

In [38]:
df_final['series_name'].unique()

array(['Access to electricity, rural (% of rural population)',
       'Access to clean fuels and technologies for cooking (% of population)',
       'Access to clean fuels and technologies for cooking, urban (% of urban population)',
       'Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)',
       'Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)',
       'Account ownership at a financial institution or with a mobile-money-service provider, poorest 40% (% of population ages 15+)'],
      dtype=object)

#  Step 11: Key Performance Indicators (KPIs)  
Now we calculate and explore high-impact development indicators from the World Bank dataset — starting with electricity access.

###  KPI 1: Average Global Electricity Access (2006–2022)  
This metric shows the overall access to electricity across all countries and years in the dataset.

In [39]:
# Filter for electricity access series (corrected)
elec = df_final[df_final['series_name'] == "Access to electricity, rural (% of rural population)"]

# Calculate global average
avg_elec = elec['value'].mean()
print(f"🌍 Average rural electricity access (2006–2022): {avg_elec:.2f}%")

🌍 Average rural electricity access (2006–2022): 78.71%


###  KPI 2: Countries with Less Than 50% Electricity Access in 2022  
This highlights regions where access to electricity is still very limited.

In [40]:
# Filter 2022 rural electricity access data
low_access_2022 = elec[elec['year'].dt.year == 2022]
low_access_2022 = low_access_2022[low_access_2022['value'] < 50]

# Display countries below 50%
low_access_2022[['country_name', 'value']].sort_values(by='value')

,country_name,value
5054,"Congo, Dem. Rep.",1.000000
5018,Chad,1.300000
5150,Equatorial Guinea,1.400000
5012,Central African Republic,1.600000
4976,Burundi,1.700000
5858,South Sudan,1.700000
4970,Burkina Faso,3.400000
5810,Sierra Leone,5.000000
5594,Mozambique,5.000000
5504,Malawi,5.600000


###  KPI 3: Percentage Improvement in Electricity Access (2006 to 2022)  
This compares each country's access to electricity in 2006 vs. 2022 to show progress.

In [41]:
# Filter for 2006 and 2022 data
elec_2006 = elec[elec['year'].dt.year == 2006].set_index('country_name')
elec_2022 = elec[elec['year'].dt.year == 2022].set_index('country_name')

# Join 2006 and 2022 on country
elec_compare = elec_2006[['value']].rename(columns={'value': 'access_2006'}).join(
    elec_2022[['value']].rename(columns={'value': 'access_2022'}),
    how='inner'
)

# Calculate % improvement
elec_compare['%_improvement'] = elec_compare['access_2022'] - elec_compare['access_2006']
elec_compare.sort_values('%_improvement', ascending=False).head(10)

,access_2006,access_2022,%_improvement
country_name,,,
Timor-Leste,21.4,100.0,78.6
Cambodia,16.1,88.0,71.9
Solomon Islands,3.9,75.4,71.5
Afghanistan,19.3,81.7,62.4
Bangladesh,40.2,99.3,59.1
Mongolia,41.5,100.0,58.5
Bhutan,42.0,100.0,58.0
Marshall Islands,42.5,100.0,57.5
Lao PDR,42.8,100.0,57.2
